In [19]:
! pip install --user sklearn

  Using cached https://files.pythonhosted.org/packages/e2/4c/6111b9a325f29527d7f262e2ee8c730d354b47a728d955e186dacad57a0d/scikit_learn-0.24.1-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/12/ec3f2e203afa394a149911729357aa48affc59c20e2c1c8297a60f33f133/threadpoolctl-2.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/34/5b/bd0f0fb5564183884d8e35b81d06d7ec06a20d1a0c8b4c407f1554691dce/joblib-1.0.0-py3-none-any.whl


In [1]:
import onnx
import torch

In [29]:
input_size = 600      
hidden1 = 64      
hidden2 = 64
hidden3 = 64
weight_bit_width = 2
act_bit_width = 2
num_classes = 1

from brevitas.nn import QuantLinear, QuantReLU, QuantIdentity
from brevitas.core.quant import QuantType
import torch.nn as nn

class QuantizedCybSecMLP(nn.Module):
    def __init__(self):
        super(QuantizedCybSecMLP, self).__init__()
        self.features = nn.Sequential(
            QuantLinear(input_size, hidden1, bias=True, weight_bit_width=weight_bit_width),
            nn.BatchNorm1d(hidden1),
            nn.Dropout(0.5),
            QuantReLU(bit_width=act_bit_width),
            QuantLinear(hidden1, hidden2, bias=True, weight_bit_width=weight_bit_width),
            nn.BatchNorm1d(hidden2),
            nn.Dropout(0.5),
            QuantReLU(bit_width=act_bit_width),
            QuantLinear(hidden2, hidden3, bias=True, weight_bit_width=weight_bit_width),
            nn.BatchNorm1d(hidden3),
            nn.Dropout(0.5),
            QuantReLU(bit_width=act_bit_width),
            QuantLinear(hidden3, num_classes, bias=True, weight_bit_width=weight_bit_width)
        )
        self.qnt_output = QuantIdentity(quant_type=QuantType.BINARY, bit_width=1, min_val=-1.0, max_val=1.0)
    
    def forward(self, x):
        x = (x + torch.tensor([1.0])) / 2.0  
        out_original = self.features(x)
        out_final = self.qnt_output(out_original)
        return out_final

brevitas_model = QuantizedCybSecMLP()

In [30]:
brevitas_model.load_state_dict(torch.load("state_dict_export.pth"), strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [60]:
import numpy as np
from torch.utils.data import TensorDataset

def get_preqnt_dataset(data_dir: str, train: bool):
    unsw_nb15_data = np.load(data_dir + "/unsw_nb15_binarized.npz")
    if train:
        partition = "train"
    else:
        partition = "test"
    part_data = unsw_nb15_data[partition].astype(np.float32)
    part_data_in = part_data[:, :-1]
    part_data_in = np.pad(part_data_in, [(0,0), (0,7)])
    part_data_in = 2*part_data_in-1
    part_data_in = torch.from_numpy(part_data_in)
    part_data_out = part_data[:, -1]
    part_data_out = 2*part_data_out-1
    part_data_out = torch.from_numpy(part_data_out)
    return TensorDataset(part_data_in, part_data_out)

test_quantized_dataset = get_preqnt_dataset(".", train=False)

In [61]:
from torch.utils.data import DataLoader, Dataset

batch_size = 1000

test_quantized_loader = DataLoader(test_quantized_dataset, batch_size=batch_size, shuffle=False)

In [62]:
from sklearn.metrics import accuracy_score

def test_padded_bipolar(model, test_loader):    
    # ensure model is in eval mode
    model.eval() 
    y_true = []
    y_pred = []
   
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            output = model(inputs.float())
            y_pred.extend(list(output.flatten()))
            y_true.extend(list(target.detach().numpy().flatten()))
        
    return accuracy_score(y_true, y_pred)

In [63]:
test_padded_bipolar(brevitas_model, test_quantized_loader)

0.918075596365933

In [46]:
import brevitas.onnx as bo

export_onnx_path = "cybsec-mlp-nids-demo.onnx"
input_shape = (1, 600)
bo.export_finn_onnx(brevitas_model, input_shape, export_onnx_path)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.


In [47]:
from finn.core.modelwrapper import ModelWrapper
from finn.core.datatype import DataType

model_file = "cybsec-mlp-nids-demo.onnx"

model = ModelWrapper(model_file)
model.set_tensor_datatype(model.graph.input[0].name, DataType.BIPOLAR)
model.save(model_file)


In [50]:
from finn.util.visualization import showInNetron

showInNetron("cybsec-mlp-nids-demo.onnx")

Serving 'cybsec-mlp-nids-demo.onnx' at http://0.0.0.0:8081


In [84]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg

model_file = "cybsec-mlp-nids-demo.onnx"

rtlsim_output_dir = "nids_demo_rtlsim"

cfg = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    default_mem_mode    = build_cfg.ComputeEngineMemMode.DECOUPLED,
    #mvau_wwidth_max     = 1000000,
    fpga_part           = "xc7z020clg400-1",
    generate_outputs=[
        build_cfg.DataflowOutputType.STITCHED_IP,
    ]
)

build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from cybsec-mlp-nids-demo.onnx
Intermediate outputs will be generated in /tmp/finn_dev_maltanar
Final outputs will be generated in nids_demo_rtlsim
Build log is at nids_demo_rtlsim/build_dataflow.log
Running step: step_tidy_up [1/16]
Running step: step_streamline [2/16]
Running step: step_convert_to_hls [3/16]
Running step: step_create_dataflow_partition [4/16]
Running step: step_target_fps_parallelization [5/16]
Running step: step_apply_folding_config [6/16]
Running step: step_generate_estimate_reports [7/16]
Running step: step_hls_codegen [8/16]
Running step: step_hls_ipgen [9/16]
Running step: step_set_fifo_depths [10/16]
Running step: step_create_stitched_ip [11/16]
Running step: step_measure_rtlsim_performance [12/16]
Running step: step_make_pynq_driver [13/16]
Running step: step_out_of_context_synthesis [14/16]
Running step: step_synthesize_bitfile [15/16]
Running step: step_deployment_package [16/16]
Completed successfully


0

In [85]:
stitched_model_filename = "nids_demo_rtlsim/intermediate_models/11_step_create_stitched_ip.onnx"
showInNetron(stitched_model_filename)


Stopping http://0.0.0.0:8081
Serving 'nids_demo_rtlsim/intermediate_models/11_step_create_stitched_ip.onnx' at http://0.0.0.0:8081


In [87]:
import os

os.environ["RTLSIM_TRACE_DEPTH"] = "4"

stitched_model_filename = "nids_demo_rtlsim/intermediate_models/11_step_create_stitched_ip.onnx"
rtlsim_model = ModelWrapper(stitched_model_filename)
rtlsim_model.set_metadata_prop("exec_mode", "rtlsim")
#rtlsim_model.set_metadata_prop("rtlsim_trace", "trace.vcd")

from finn.util.basic import gen_finn_dt_tensor

idt = rtlsim_model.get_tensor_datatype("global_in")
r_inp = gen_finn_dt_tensor(idt, (1, 600))
in_dict = {"global_in" : r_inp}
out_dict = execute_onnx(rtlsim_model, in_dict)

%Warning-STMTDLY: /tmp/finn_dev_maltanar/vivado_stitch_proj_f1gbdape/finn_design_wrapper.v:797: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: Use "/* verilator lint_off STMTDLY */" and lint_on around source to disable this message.
%Warning-STMTDLY: /tmp/finn_dev_maltanar/vivado_stitch_proj_f1gbdape/finn_design_wrapper.v:798: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_maltanar/vivado_stitch_proj_f1gbdape/finn_design_wrapper.v:799: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_maltanar/vivado_stitch_proj_f1gbdape/finn_design_wrapper.v:800: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_maltanar/vivado_stitch_proj_f1gbdape/finn_design_wrapper.v:801: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_maltanar/vivado_stitch_proj_f1gbdape/finn_design_wrapper.v:3456: Unsupported: Ignoring delay on this dela

make: Entering directory '/tmp/finn_dev_maltanar/pyverilator_ipstitched_wn_4xo96'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_TRACE=1 -DVM_COVERAGE=0 -Wno-char-subscripts -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_maltanar/pyverilator_ipstitched_wn_4xo96/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_TRACE=1 -DVM_COVERAGE=0 -Wno-char-subscripts -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_TRACE=1 -DVM_COVER

ar: creating Vfinn_design_wrapper__ALL.a


In [98]:
from sklearn.metrics import accuracy_score
from finn.core.rtlsim_exec import rtlsim_exec
from tqdm import tqdm, trange

def test_padded_bipolar_finn(finn_model, test_loader):
    t = trange(len(test_quantized_dataset), desc="Validation", leave=True)
    y_true = []
    y_pred = []
   
    with torch.no_grad():
        for data,tt in zip(test_loader,t):
            inputs, target = data
            in_dict = {"global_in" : inputs.float().numpy()}
            out_dict = rtlsim_exec(finn_model, in_dict)
            output = in_dict["global_out"]
            y_pred.extend(list(output.flatten()))
            y_true.extend(list(target.detach().numpy().flatten()))
            t.set_description("accuracy = %f" % accuracy_score(y_true, y_pred))
            t.refresh() # to show immediately the update           
        
    return accuracy_score(y_true, y_pred)

In [99]:
test_quantized_loader_b1 = DataLoader(test_quantized_dataset, batch_size=10, shuffle=False)
test_padded_bipolar_finn(rtlsim_model, test_quantized_loader_b1)















Validation:   0%|          | 0/82332 [00:00<?, ?it/s]













accuracy = 0.100000:   0%|          | 0/82332 [00:00<?, ?it/s]













accuracy = 0.100000:   0%|          | 0/82332 [00:00<?, ?it/s]













accuracy = 0.100000:   0%|          | 1/82332 [00:00<5:30:02,  4.16it/s]













accuracy = 0.100000:   0%|          | 1/82332 [00:00<5:30:02,  4.16it/s]













accuracy = 0.100000:   0%|          | 1/82332 [00:00<5:30:02,  4.16it/s]













accuracy = 0.100000:   0%|          | 2/82332 [00:00<5:28:05,  4.18it/s]













accuracy = 0.133333:   0%|          | 2/82332 [00:01<5:28:05,  4.18it/s]













accuracy = 0.133333:   0%|          | 2/82332 [00:01<5:28:05,  4.18it/s]













accuracy = 0.133333:   0%|          | 3/82332 [00:01<12:12:58,  1.87it/s]













accuracy = 0.100000:   0%|          | 3/82332 [00:01<12:12:58,  1.87it/s]













accuracy = 0.100000:   0%|          | 3/82332 [00:01<12:12:58,  1.87it/s]





accuracy = 0.038095:   0%|          | 21/82332 [00:05<5:29:28,  4.16it/s]













accuracy = 0.040909:   0%|          | 21/82332 [00:06<5:29:28,  4.16it/s]













accuracy = 0.040909:   0%|          | 21/82332 [00:06<5:29:28,  4.16it/s]













accuracy = 0.040909:   0%|          | 22/82332 [00:06<5:26:53,  4.20it/s]













accuracy = 0.052174:   0%|          | 22/82332 [00:06<5:26:53,  4.20it/s]













accuracy = 0.052174:   0%|          | 22/82332 [00:06<5:26:53,  4.20it/s]













accuracy = 0.052174:   0%|          | 23/82332 [00:06<5:26:48,  4.20it/s]













accuracy = 0.058333:   0%|          | 23/82332 [00:06<5:26:48,  4.20it/s]













accuracy = 0.058333:   0%|          | 23/82332 [00:06<5:26:48,  4.20it/s]













accuracy = 0.058333:   0%|          | 24/82332 [00:06<5:25:18,  4.22it/s]













accuracy = 0.060000:   0%|          | 24/82332 [00:06<5:25:18,  4.22it/s]













accuracy = 0.060000:   0%|          | 24/82

accuracy = 0.176190:   0%|          | 42/82332 [00:12<5:31:16,  4.14it/s]













accuracy = 0.172093:   0%|          | 42/82332 [00:12<5:31:16,  4.14it/s]













accuracy = 0.172093:   0%|          | 42/82332 [00:12<5:31:16,  4.14it/s]













accuracy = 0.172093:   0%|          | 43/82332 [00:12<5:29:29,  4.16it/s]













accuracy = 0.170455:   0%|          | 43/82332 [00:12<5:29:29,  4.16it/s]













accuracy = 0.170455:   0%|          | 43/82332 [00:12<5:29:29,  4.16it/s]













accuracy = 0.170455:   0%|          | 44/82332 [00:12<5:28:45,  4.17it/s]













accuracy = 0.166667:   0%|          | 44/82332 [00:12<5:28:45,  4.17it/s]













accuracy = 0.166667:   0%|          | 44/82332 [00:12<5:28:45,  4.17it/s]













accuracy = 0.166667:   0%|          | 45/82332 [00:12<5:28:23,  4.18it/s]













accuracy = 0.165217:   0%|          | 45/82332 [00:12<5:28:23,  4.18it/s]













accuracy = 0.165217:   0%|          | 45/82

accuracy = 0.139683:   0%|          | 63/82332 [00:17<5:35:07,  4.09it/s]













accuracy = 0.137500:   0%|          | 63/82332 [00:18<5:35:07,  4.09it/s]













accuracy = 0.137500:   0%|          | 63/82332 [00:18<5:35:07,  4.09it/s]













accuracy = 0.137500:   0%|          | 64/82332 [00:18<5:37:04,  4.07it/s]













accuracy = 0.135385:   0%|          | 64/82332 [00:18<5:37:04,  4.07it/s]













accuracy = 0.135385:   0%|          | 64/82332 [00:18<5:37:04,  4.07it/s]













accuracy = 0.135385:   0%|          | 65/82332 [00:18<5:32:19,  4.13it/s]













accuracy = 0.136364:   0%|          | 65/82332 [00:18<5:32:19,  4.13it/s]













accuracy = 0.136364:   0%|          | 65/82332 [00:18<5:32:19,  4.13it/s]













accuracy = 0.136364:   0%|          | 66/82332 [00:18<5:34:40,  4.10it/s]













accuracy = 0.137313:   0%|          | 66/82332 [00:18<5:34:40,  4.10it/s]













accuracy = 0.137313:   0%|          | 66/82

accuracy = 0.136905:   0%|          | 84/82332 [00:24<5:51:42,  3.90it/s]













accuracy = 0.142353:   0%|          | 84/82332 [00:24<5:51:42,  3.90it/s]













accuracy = 0.142353:   0%|          | 84/82332 [00:24<5:51:42,  3.90it/s]













accuracy = 0.142353:   0%|          | 85/82332 [00:24<5:46:07,  3.96it/s]













accuracy = 0.141860:   0%|          | 85/82332 [00:24<5:46:07,  3.96it/s]













accuracy = 0.141860:   0%|          | 85/82332 [00:24<5:46:07,  3.96it/s]













accuracy = 0.141860:   0%|          | 86/82332 [00:24<5:43:59,  3.98it/s]













accuracy = 0.142529:   0%|          | 86/82332 [00:24<5:43:59,  3.98it/s]













accuracy = 0.142529:   0%|          | 86/82332 [00:24<5:43:59,  3.98it/s]













accuracy = 0.142529:   0%|          | 87/82332 [00:24<5:41:17,  4.02it/s]













accuracy = 0.144318:   0%|          | 87/82332 [00:24<5:41:17,  4.02it/s]













accuracy = 0.144318:   0%|          | 87/82

...

accuracy = 0.148052:   0%|          | 231/82332 [01:05<5:40:18,  4.02it/s]













accuracy = 0.148276:   0%|          | 231/82332 [01:05<5:40:18,  4.02it/s]













accuracy = 0.148276:   0%|          | 231/82332 [01:05<5:40:18,  4.02it/s]













accuracy = 0.148276:   0%|          | 232/82332 [01:05<5:37:27,  4.05it/s]













accuracy = 0.148069:   0%|          | 232/82332 [01:06<5:37:27,  4.05it/s]













accuracy = 0.148069:   0%|          | 232/82332 [01:06<5:37:27,  4.05it/s]













accuracy = 0.148069:   0%|          | 233/82332 [01:06<5:33:16,  4.11it/s]













accuracy = 0.147863:   0%|          | 233/82332 [01:06<5:33:16,  4.11it/s]













accuracy = 0.147863:   0%|          | 233/82332 [01:06<5:33:16,  4.11it/s]













accuracy = 0.147863:   0%|          | 234/82332 [01:06<5:32:38,  4.11it/s]













accuracy = 0.147234:   0%|          | 234/82332 [01:06<5:32:38,  4.11it/s]













accuracy = 0.147234:   0%|      

accuracy = 0.143254:   0%|          | 252/82332 [01:11<6:00:04,  3.80it/s]













accuracy = 0.144664:   0%|          | 252/82332 [01:12<6:00:04,  3.80it/s]













accuracy = 0.144664:   0%|          | 252/82332 [01:12<6:00:04,  3.80it/s]













accuracy = 0.144664:   0%|          | 253/82332 [01:12<5:50:45,  3.90it/s]













accuracy = 0.144488:   0%|          | 253/82332 [01:12<5:50:45,  3.90it/s]













accuracy = 0.144488:   0%|          | 253/82332 [01:12<5:50:45,  3.90it/s]













accuracy = 0.144488:   0%|          | 254/82332 [01:12<5:43:54,  3.98it/s]













accuracy = 0.145098:   0%|          | 254/82332 [01:12<5:43:54,  3.98it/s]













accuracy = 0.145098:   0%|          | 254/82332 [01:12<5:43:54,  3.98it/s]













accuracy = 0.145098:   0%|          | 255/82332 [01:12<5:39:43,  4.03it/s]













accuracy = 0.145313:   0%|          | 255/82332 [01:12<5:39:43,  4.03it/s]













accuracy = 0.145313:   0%|      

accuracy = 0.142857:   0%|          | 273/82332 [01:18<6:19:35,  3.60it/s]













accuracy = 0.143066:   0%|          | 273/82332 [01:18<6:19:35,  3.60it/s]













accuracy = 0.143066:   0%|          | 273/82332 [01:18<6:19:35,  3.60it/s]













accuracy = 0.143066:   0%|          | 274/82332 [01:18<6:02:59,  3.77it/s]













accuracy = 0.142545:   0%|          | 274/82332 [01:18<6:02:59,  3.77it/s]













accuracy = 0.142545:   0%|          | 274/82332 [01:18<6:02:59,  3.77it/s]













accuracy = 0.142545:   0%|          | 275/82332 [01:18<5:51:04,  3.90it/s]













accuracy = 0.142391:   0%|          | 275/82332 [01:18<5:51:04,  3.90it/s]













accuracy = 0.142391:   0%|          | 275/82332 [01:18<5:51:04,  3.90it/s]













accuracy = 0.142391:   0%|          | 276/82332 [01:18<5:44:07,  3.97it/s]













accuracy = 0.141877:   0%|          | 276/82332 [01:19<5:44:07,  3.97it/s]













accuracy = 0.141877:   0%|      

accuracy = 0.137415:   0%|          | 294/82332 [01:24<9:32:58,  2.39it/s] 













accuracy = 0.136949:   0%|          | 294/82332 [01:24<9:32:58,  2.39it/s]













accuracy = 0.136949:   0%|          | 294/82332 [01:24<9:32:58,  2.39it/s]













accuracy = 0.136949:   0%|          | 295/82332 [01:24<8:21:25,  2.73it/s]













accuracy = 0.136486:   0%|          | 295/82332 [01:24<8:21:25,  2.73it/s]













accuracy = 0.136486:   0%|          | 295/82332 [01:24<8:21:25,  2.73it/s]













accuracy = 0.136486:   0%|          | 296/82332 [01:24<7:28:33,  3.05it/s]













accuracy = 0.136364:   0%|          | 296/82332 [01:24<7:28:33,  3.05it/s]













accuracy = 0.136364:   0%|          | 296/82332 [01:24<7:28:33,  3.05it/s]













accuracy = 0.136364:   0%|          | 297/82332 [01:24<6:50:43,  3.33it/s]













accuracy = 0.136242:   0%|          | 297/82332 [01:25<6:50:43,  3.33it/s]













accuracy = 0.136242:   0%|     

KeyboardInterrupt: 